In [ ]:
try:
    get_ipython
    isnotebook = True
except:
    isnotebook = False

import os
import sys
import numpy as np
import scipy as sp
import matplotlib
if not isnotebook:
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
import time
from qutip import *

In [ ]:
import qocttools
import qocttools.models.GdW30 as GdW30
import qocttools.hamiltonians as hamiltonians
import qocttools.math_extra as math_extra
import qocttools.pulses as pulses
import qocttools.qoct as qoct
import qocttools.solvers as solvers

In [ ]:
qocttools.about()

In [ ]:
data = []

# Introduction

In this notebook we benchmark various propagation methods, using the GdW$_{30}$ model.

# Model

The model is defined by the Hamiltonian:

\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
where the time-independent part is given by:
\begin{equation}
        \hat{H}_0 = D\bigg[\hat{S}_z^2 - \frac{1}{3}S(S + 1)\bigg] + E[\hat{S}_x^2 - \hat{S}_y^2] - g\mu_B\hat{\vec{S}}\cdot\vec{H}
\end{equation}
and the time-dependent part is:
\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
The perturbation is a magnetic field:
\begin{equation}
        \hat{V} = -g\mu_B\hat{\vec{S}}\cdot\vec{H}_m 
\end{equation}

In this case:

* $S = 7/2$

* $D$ = 1281 MHz

* $E$ = 294 MHz

* $\vec{H} = (0.0, 0.0, 0.650)$ T

* $\vec{H}_m = (0, 0.001, 0)$ T

In [ ]:
# Parameters definition
S = 7/2 # spin
E = 294 # value in MHz
D = 1281 # value in MHz
dim = int(2*S + 1) #matrix dim

In [ ]:
H = np.array([0.15, 0, 0.0], dtype = float) #magnetic field in T
H_m = np.array([0, 0.001, 0], dtype = float) #only in presence of perturbation (T)
H0 = GdW30.hGdW30(D, E, H)
V = GdW30.vGdW30(H_m)

In [ ]:
eigenvalues, eigenstates = H0.eigenstates()

In [ ]:
P0 = eigenstates[0] * eigenstates[0].dag()
P1 = eigenstates[1] * eigenstates[1].dag()
H0d = H0.transform(eigenstates)
Vd = V.transform(eigenstates)
P0d = basis(dim, 0) * basis(dim, 0).dag()
P1d = basis(dim, 1) * basis(dim, 1).dag()

In [ ]:
w = np.zeros(dim-1)
for i in range(int(dim - 1)):
    #w.append(eigenvalues[i+1] - eigenvalues[i]) #in MHz
    w[i] = eigenvalues[i+1] - eigenvalues[i]

# We will define a "characteristic period" of the field-free Hamiltonian
# as the period corresponding to the maximum frequency. Note that we are
# considering only the "neighbour energies" to define the frequencies.
tau0 = (2*np.pi/np.max(np.array(w)))
tau = (2*np.pi/w[0])

We will apply a $\pi$-pulse; lambda_ is the amplitude (in mT), and mu is the coupling matrix element corresponding to the first transition.

The form of the amplitude td function is:
\begin{equation}
g(t) = \lambda \sin(\omega_0 t)
\end{equation}

In [ ]:
lambda_ = 1.0
mu = np.abs(Vd.full()[0, 1])
T = 0.5*np.pi/(lambda_ * mu)

In [ ]:
def gu(t, u):
    return u[0] * np.sin(u[1]*t)

def dgdu(t, u, m):
    if m == 0:
        return np.sin(u[1]*t)
    else:
        return u[0] * np.cos(u[1]*t)

In [ ]:
u = np.array([lambda_, w[0]])
f = pulses.pulse("user_defined", T, u = u)
f.assign_user_defined_function(gu, dgdu)

In [ ]:
def benchmark(method, H, f, psi0, tfactors, finalst, picture = 'schrodinger'):
    if H.function:
        #args = { "f": [f[l].fu for l in range(len(f))] }
        args = { "f": [f.fu]}
        H0 = H.H0(0.0, args)
        method = 'sesolve'
        picture = 'schrodinger'
    else:
        H0 = H.H0
    diff = np.zeros(tfactors.size)
    cputimes = np.zeros(tfactors.size)
    dt = np.zeros(tfactors.size)
    if picture == 'interaction':
        interaction_picture = True
    else:
        interaction_picture = False
    i = 0
    for tfactor in tfactors:
        times = math_extra.timegrid(H0, T, tfactor)
        t0 = time.time()
        result = solvers.solve(method, H, f, psi0, times,
                               returnQoutput = False,
                               interaction_picture = interaction_picture)
        tf = time.time()
        cputimes[i] = tf-t0
        dt[i] = times[1]
        finalstapprox = Qobj(result[-1])
        diff[i] = (finalstapprox-finalst).norm()
        print(i, dt[i], diff[i], cputimes[i])
        i = i + 1
    return dt, diff, cputimes

In [ ]:
def plot_results(dt, diff, times):
    a = np.polyfit(np.log10(dt), np.log10(diff), deg = 1)

    fig, ax = plt.subplots(1, 2, figsize = (12, 4))
    ax[0].plot(np.log10(dt), np.log10(diff), "bo")
    ax[0].plot(np.log10(dt), a[1]+a[0]*np.log10(dt), 
               label = "y = {:5.2f}+{:5.2f}x".format(a[1], a[0]))
    ax[0].set_xlabel(r"$\log_{10}\Delta t$")
    ax[0].set_ylabel(r"$\log_{10}$ Error")
    ax[0].legend(loc = 'best')
    ax[1].set_xlabel(r"$\log_{10}$ Error")
    ax[1].set_ylabel(r"$\log_{10}$ Cost")
    ax[1].plot(np.log10(diff), np.log10(times), "bo-")
    fig.tight_layout(pad = 2.0)
    if isnotebook:
        plt.show()

# Reference run

In [ ]:
theta = 0.01
times = math_extra.timegrid(H0, T, theta)
print("Number of time steps =", times.size)
print("dt =", times[1])
print("T =", times[-1])

In [ ]:
u = np.array([lambda_, w[0]])
f = pulses.pulse("user_defined", T, u = u)
f.assign_user_defined_function(gu, dgdu)

In [ ]:
ut = f.fu(times)
ft = pulses.pulse("realtime", T, u = ut)

## State propagation

In [ ]:
psi0 = eigenstates[0]

In [ ]:
t0 = time.time()
H = hamiltonians.hamiltonian(H0, V)
result = solvers.solve('rk4', H, f, psi0, times,
                       returnQoutput = False)
t1 = time.time()
print("Elapsed time =", t1-t0)

In [ ]:
p1 = np.zeros(times.shape[0])
for j in range(times.shape[0]):
    p1[j] = expect(P1, Qobj(result[j]))

In [ ]:
finalst = Qobj(result[-1])
print("P1(T) =", expect(P1, finalst))

In [ ]:
finalstp = (finalst.copy()).transform(eigenstates)
finalstpint = (1j*T*H0d).expm() * finalstp

In [ ]:
fig, ax = plt.subplots()
ax.plot(times, p1)
if isnotebook:
    plt.show()

## Propagator propagation

In [ ]:
U0 = qeye(dim)

In [ ]:
t0 = time.time()
resultU = solvers.solve('rk4', hamiltonians.hamiltonian(H0, V), f, U0, times,
                       returnQoutput = False)
t1 = time.time()
print("Elapsed time =", t1-t0)

In [ ]:
finalU = Qobj(resultU[-1])

In [ ]:
print(expect(P1, finalU*psi0))

# RK4, Schrödinger picture

In [ ]:
tfactors = np.array([theta*80, theta*40, theta*20, theta*10])

In [ ]:
dt, diff, cputimes = benchmark('rk4', hamiltonians.hamiltonian(H0, V), f, eigenstates[0], tfactors, finalst)
data.append(diff[0])

In [ ]:
plot_results(dt, diff, cputimes)

# Internal qutip solver, Schrödinger picture, Hamiltonian-as-a-function

In [ ]:
tfactors = np.array([theta*80, theta*40, theta*20, theta*10])

In [ ]:
def Hfunc(t, args):
    return H0 + args["f"][0](t) * V
def Vfunc(t, args):
    return V
H_ = hamiltonians.hamiltonian(Hfunc, Vfunc)

In [ ]:
dt, diff, cputimes = benchmark('sesolve', H_, f, eigenstates[0], tfactors, finalst)
data.append(diff[0])

In [ ]:
plot_results(dt, diff, cputimes)

# RK4, Schrödinger picture, "realtime" pulse

In [ ]:
tfactors = np.array([theta*80, theta*40, theta*20, theta*10])

In [ ]:
dt, diff, cputimes = benchmark('rk4', hamiltonians.hamiltonian(H0, V), ft, eigenstates[0], tfactors, finalst)
data.append(diff[0])

In [ ]:
plot_results(dt, diff, cputimes)

# RK4, Schrödinger picture; propagator

In [ ]:
tfactors = np.array([theta*80, theta*40, theta*20, theta*10])

In [ ]:
dt, diff, cputimes = benchmark('rk4', hamiltonians.hamiltonian(H0, V), f, U0, tfactors, finalU)
data.append(diff[0])

In [ ]:
plot_results(dt, diff, cputimes)

# RK4, Schrödinger picture, eigenbasis, elimination of first frequency

We will now do the same calculation, but use the eigenbasis, and substract the first frequency.

In [ ]:
tfactors = np.array([theta*160, theta*80, theta*40, theta*20])

In [ ]:
finalstpp = finalstp.copy()
for j in range(dim):
    finalstpp.data[j, 0] = np.exp(1j * T * eigenvalues[0]) * finalstpp.data[j, 0]

In [ ]:
dt, diff, cputimes = benchmark('rk4', hamiltonians.hamiltonian(H0d-eigenvalues[0], Vd),
                               f, basis(dim, 0), tfactors, finalstpp)
data.append(diff[0])

In [ ]:
plot_results(dt, diff, cputimes)

# RK4, interaction picture

In [ ]:
tfactors = np.array([theta*1000, theta*500, theta*250, theta*125])

In [ ]:
dt, diff, cputimes = benchmark('rk4', hamiltonians.hamiltonian(H0d, Vd), f, basis(dim, 0), tfactors, finalstpint,
                               picture = 'interaction')
data.append(diff[0])

In [ ]:
plot_results(dt, diff, cputimes)

# EMR, Schrödinger picture, eigenbasis, elimination of first frequency

In [ ]:
tfactors = np.array([theta*160, theta*80, theta*40, theta*20])

In [ ]:
finalstpp = finalstp.copy()
for j in range(dim):
    finalstpp.data[j, 0] = np.exp(1j * T * eigenvalues[0]) * finalstpp.data[j, 0]

In [ ]:
dt, diff, cputimes = benchmark('cfmagnus2', hamiltonians.hamiltonian(H0d-eigenvalues[0], Vd),
                               f, basis(dim, 0), tfactors, finalstpp)
data.append(diff[0])

In [ ]:
plot_results(dt, diff, cputimes)

# EMR, interaction picture

In [ ]:
tfactors = np.array([theta*400, theta*200, theta*100, theta*50])

In [ ]:
dt, diff, cputimes = benchmark('cfmagnus2', hamiltonians.hamiltonian(H0d, Vd),
                               f, basis(dim, 0), tfactors, finalstpint,
                               picture = 'interaction')
data.append(diff[0])

In [ ]:
plot_results(dt, diff, cputimes)

# CF4, Schrödinger picture, eigenbasis, elimination of the first frequency

In [ ]:
tfactors = np.array([theta*1000, theta*500, theta*250, theta*125])

In [ ]:
finalstpp = finalstp.copy()
for j in range(dim):
    finalstpp.data[j, 0] = np.exp(1j * T * eigenvalues[0]) * finalstpp.data[j, 0]

In [ ]:
dt, diff, cputimes = benchmark('cfmagnus4', hamiltonians.hamiltonian(H0d-eigenvalues[0], Vd),
                               f, basis(dim, 0), tfactors, finalstpp)
data.append(diff[0])

In [ ]:
plot_results(dt, diff, cputimes)

# CF4, interaction picture

In [ ]:
tfactors = np.array([theta*1000, theta*500, theta*250, theta*125])

In [ ]:
dt, diff, cputimes = benchmark('cfmagnus4', hamiltonians.hamiltonian(H0d, Vd),
                               f,basis(dim, 0), tfactors, finalstpint,
                               picture = 'interaction')
data.append(diff[0])

In [ ]:
plot_results(dt, diff, cputimes)

In [ ]:
with open("data", "w") as f:
    for i in data:
        f.write("{:.14e}\n".format(i))